<a href="https://colab.research.google.com/github/gvsmaneesha/t5_training/blob/main/Training_Instance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd ~/../content

/content


In [2]:
!rm -rf t5_training
!git clone https://github.com/gvsmaneesha/t5_training.git

Cloning into 't5_training'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 108 (delta 44), reused 77 (delta 24), pack-reused 0
Receiving objects: 100% (108/108), 4.25 MiB | 9.54 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [3]:
cd t5_training

/content/t5_training


In [4]:
ls -l

total 104
drwxr-xr-x 2 root root  4096 Feb 22 08:35 config/
drwxr-xr-x 2 root root  4096 Feb 22 08:35 datasets/
drwxr-xr-x 3 root root  4096 Feb 22 08:35 learning_modules/
drwxr-xr-x 3 root root  4096 Feb 22 08:35 lightning_logs/
-rw-r--r-- 1 root root  2072 Feb 22 08:35 main.py
-rw-r--r-- 1 root root    47 Feb 22 08:35 README.md
-rw-r--r-- 1 root root   102 Feb 22 08:35 requirements.txt
-rw-r--r-- 1 root root 73811 Feb 22 08:35 Training_Instance.ipynb


In [5]:
!pip install -r requirements.txt

     |████████████████████████████████| 3.1 MB 4.0 MB/s 
     |████████████████████████████████| 1.2 MB 42.7 MB/s 
     |████████████████████████████████| 64 kB 1.3 MB/s 
     |████████████████████████████████| 752.0 MB 9.3 kB/s 
     |████████████████████████████████| 816 kB 50.3 MB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 3.3 MB 43.8 MB/s 
     |████████████████████████████████| 895 kB 54.0 MB/s 
     |████████████████████████████████| 596 kB 48.8 MB/s 
     |████████████████████████████████| 829 kB 50.3 MB/s 
     |████████████████████████████████| 133 kB 52.6 MB/s 
     |████████████████████████████████| 1.1 MB 52.5 MB/s 
     |████████████████████████████████| 144 kB 61.5 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 271 kB 72.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=1014893423c39f56b3ff231ad5dc42eb281c676c8104512ef

# New Section

In [6]:
print("imported all modules...")
import yaml
from pandas import read_csv
import pytorch_lightning  as pl
import argparse
import numpy as np
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics
from torch.utils.data import Dataset, DataLoader
from learning_modules.t5_tuning_modules import  T5Tokenizer,TweetDataset,LoggingCallback,T5FineTuner
from learning_modules.data_utils import create_dataset
print("successfully imported all modules")

with open('config/configuration.yaml',"r") as f:
        config = yaml.load(f,Loader=yaml.FullLoader)

print(config)
config["args_dict"]["learning_rate"] = float(config["args_dict"]["learning_rate"])
config["args_dict"]["adam_epsilon"] = float(config["args_dict"]["adam_epsilon"])

imported all modules...


ModuleNotFoundError: ignored

In [7]:
create_dataset("tweet_dataset.csv")

training data import
Created dataset !!


(           textID  ...  sentiment
 11850  1963371547  ...      worry
 34804  1752963154  ...  happiness
 21566  1694257220  ...    neutral
 33741  1752698535  ...        fun
 4291   1960316102  ...      worry
 ...           ...  ...        ...
 23028  1694533660  ...  happiness
 25733  1695190557  ...  happiness
 36420  1753310243  ...  happiness
 22652  1694473432  ...      worry
 11920  1963393635  ...      worry
 
 [16660 rows x 4 columns],            textID  ... sentiment
 10784  1963014451  ...   sadness
 10049  1962810142  ...   sadness
 26386  1695376838  ...  surprise
 16034  1965098072  ...     anger
 14213  1964385036  ...   sadness
 ...           ...  ...       ...
 2622   1957588074  ...   neutral
 24384  1694833862  ...   sadness
 8895   1962365221  ...      hate
 19394  1966250104  ...     worry
 27899  1695964120  ...       fun
 
 [5554 rows x 4 columns],            textID  ...   sentiment
 31500  1751955900  ...   happiness
 15482  1964850830  ...  enthusiasm
 27560  1

In [8]:
#token initializer
tokenizer = T5Tokenizer.from_pretrained('t5-base')

#load dataset
dataset = TweetDataset(tokenizer, data_dir='datasets/', type_path='val')
print("Length of dataset is :",len(dataset))

data = dataset[69]
print(tokenizer.decode(data['source_ids']))
print(tokenizer.decode(data['target_ids']))
args = argparse.Namespace(**config["args_dict"])

checkpoint_callback = pl.callbacks.ModelCheckpoint(
                dirpath=args.output_dir, monitor="val_loss", mode="min", save_top_k=5
        )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


['textID', 'text', 'selected_text', 'sentiment']


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:191: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


Length of dataset is : 5553
text: lol, I<unk> ve just realised that i<unk> ve been sending messages that were meant to be to you, to a page that doesn<unk> t exist sentiment: worry</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

In [9]:
train_params = config["train_params"]
train_params["callbacks"] = [LoggingCallback()]
train_params["checkpoint_callback"]=checkpoint_callback
print(train_params)

{'accumulate_grad_batches': 1, 'gpus': 0, 'max_epochs': 1, 'precision': 32, 'amp_level': 'O1', 'gradient_clip_val': 1.0, 'callbacks': [<learning_modules.t5_tuning_modules.LoggingCallback object at 0x7f4b0457a3d0>], 'checkpoint_callback': <pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f4b04ac7190>}


In [10]:
# Model Intializer
model = T5FineTuner(args)

#trainer
trainer = pl.Trainer(**train_params)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores


In [12]:
%env JOBLIB_TEMP_FOLDER=/tmp

env: JOBLIB_TEMP_FOLDER=/tmp


In [13]:
trainer.fit(model)


  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='

['textID', 'text', 'selected_text', 'sentiment']


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The validation_epoch_end should not return anything as of 9.1. To log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.
# log on a step or aggregate epoch metric to the logger and/or progress bar (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The {progress_bar:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.
# log on a step or aggregate epoch metri

['textID', 'text', 'selected_text', 'sentiment']


Training: 0it [00:00, ?it/s]

RuntimeError: ignored

In [ ]:
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics

In [ ]:
dataset =  TweetDataset(tokenizer, data_dir='datasets/', type_path='val')
loader = DataLoader(dataset, batch_size=32, num_workers=4)

In [ ]:
model.model.eval()
outputs = []
targets = []
for batch in tqdm(loader):
  outs = model.model.generate(input_ids=batch['source_ids'].cuda(), 
                              attention_mask=batch['source_mask'].cuda(), 
                              max_length=200)

  dec = [tokenizer.decode(ids) for ids in outs]
  target = [tokenizer.decode(ids) for ids in batch["target_ids"]]
  
  outputs.extend(dec)
  targets.extend(target)